In [3]:
# import library for Support Vector Classification
from sklearn.svm import SVC

# import library for plot
import matplotlib.pyplot as plt

# import library for data handling
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math

# import library for pytorch
import torch
import torch.nn as nn

# import custom function
from stockLSTM import SequenceDataset, ShallowRegressionLSTM, train, test, get_stock_data, split_train_test, predict

# Get cuda device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# predict df
predict_df = pd.DataFrame(columns = ["High", "Low", "Open", "Close"])

In [4]:
# Get the past stock price
df = get_stock_data(stock_name = 'TSLA', source = 'yahoo', past_year = 2)

# Split data into train and test
train_df, test_df, feature, target, train_size, test_size, target_mean, target_std = split_train_test(training_target = "Close", df = df, train_ratio = 0.8)


In [5]:
# create dataset and data loader
batch_size = 4
seq_len = 60

torch.manual_seed(seq_len*batch_size/2)

train_dataset = SequenceDataset(train_df, target, feature, seq_len)
test_dataset = SequenceDataset(test_df, target, feature, seq_len)
train_looader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# get the first batch and its shape
x, y = next(iter(train_looader))
print("Features shape: ", x.shape)
print("Target shape: ", y.shape)

Features shape:  torch.Size([4, 60, 5])
Target shape:  torch.Size([4])


In [8]:
# create model, loss and optimizer
learning_rate = 5e-5
num_hidden = 16
epochs = 40

model = ShallowRegressionLSTM(n_features = len(feature), n_hidden = num_hidden, device=device).to(device)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# train and test model
print("Untrained test\n----------------")
test(model, test_loader, loss, device)
print("\nTraining\n----------------")

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n")
    train(model, train_looader, loss, optimizer, device)
    test(model, test_loader, loss, device)

Untrained test
----------------
Test loss: 0.2929

Training
----------------
Epoch 1

Train loss: 0.9970
Test loss: 0.2708
Epoch 2

Train loss: 0.9396
Test loss: 0.2510
Epoch 3

Train loss: 0.8805
Test loss: 0.2326
Epoch 4

Train loss: 0.8161
Test loss: 0.2150
Epoch 5

Train loss: 0.7609
Test loss: 0.1988
Epoch 6

Train loss: 0.6948
Test loss: 0.1821
Epoch 7

Train loss: 0.6333
Test loss: 0.1668
Epoch 8

Train loss: 0.5770
Test loss: 0.1528
Epoch 9

Train loss: 0.5267
Test loss: 0.1392
Epoch 10

Train loss: 0.4859
Test loss: 0.1270
Epoch 11

Train loss: 0.4357
Test loss: 0.1157
Epoch 12

Train loss: 0.4006
Test loss: 0.1048
Epoch 13

Train loss: 0.3666
Test loss: 0.0952
Epoch 14

Train loss: 0.3374
Test loss: 0.0860
Epoch 15

Train loss: 0.3113
Test loss: 0.0777
Epoch 16

Train loss: 0.2858
Test loss: 0.0705
Epoch 17

Train loss: 0.2604
Test loss: 0.0644
Epoch 18

Train loss: 0.2388
Test loss: 0.0599
Epoch 19

Train loss: 0.2149
Test loss: 0.0575
Epoch 20

Train loss: 0.1956
Test loss:

In [11]:
train_eval = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

train_df["prediction"] = predict(train_eval, model, device)
test_df["prediction"] = predict(test_loader, model, device)

df_out = pd.concat([train_df, test_df], axis=0)[[target, "prediction"]]

for c in df_out.columns:
    df_out[c] = df_out[c] * target_std + target_mean




Index(['Close_t+1', 'prediction'], dtype='object')